In [2]:
from shiptrack import get_data, get_preprocessing, losses, fit_model
from segmentation_models import get_preprocessing
from segmentation_models import Unet
from segmentation_models.metrics import iou_score

from keras.optimizers import Adam
import tensorflow as tf



Segmentation Models: using `tf.keras` framework.
channels_last


In [3]:
# Hyperparams
epochs = 30
batch_size = 8
learning_rate = 0.01
augment = False
encoder_freeze = False
backbone = "resnet152"
test_prop = 5
loss = "bce_jaccard_loss"

INT_IMG_SIZE = (2240, 1344)
IMG_SIZE = 448

In [3]:
from datetime import datetime
now = datetime.now()

model_name = f"{now.strftime("%Y%m%d_%H%M%S")}_new_{backbone}_{loss}{'_augmented' if augment else ''}"
# System paths
training_dir = "/lustre_scratch/duncanwp/combined_v3_typed_new_composite"
tensorboard_dir = f"/lustre_scratch/duncanwp/tensorboard/{model_name}"
model_dir = f"/lustre_scratch/duncanwp/models/{model_name}"

In [4]:
def get_generator(all_data, all_labels):

    for data, labels in zip(all_data, all_labels):
#         print(data)
        # Resize the data
        _data = tf.image.resize(data, INT_IMG_SIZE) / 255.
        _labels = tf.image.resize(tf.expand_dims(labels, -1), INT_IMG_SIZE, 'nearest') # Adding an extra color dim for tf.image
#         print(_data)
#         print(_labels)

        # Slice the images to the final size...
        flat_patches = tf.image.extract_patches(images=tf.expand_dims(_data, axis=0),
                                                sizes=[1, IMG_SIZE, IMG_SIZE, 1],
                                                strides=[1, IMG_SIZE, IMG_SIZE, 1],  # This should be the same as sizes
                                                rates=[1, 1, 1, 1],
                                                padding='VALID')
        _data = tf.reshape(flat_patches, [-1, IMG_SIZE, IMG_SIZE, 3])  # Stack them along the leading dim

        # ...And the labels
        flat_patches = tf.image.extract_patches(images=tf.expand_dims(_labels, axis=0),
                                                sizes=[1, IMG_SIZE, IMG_SIZE, 1],
                                                strides=[1, IMG_SIZE, IMG_SIZE, 1],  # This should be the same as sizes
                                                rates=[1, 1, 1, 1],
                                                padding='VALID')
        _labels = tf.reshape(flat_patches, [-1, IMG_SIZE, IMG_SIZE])  # Stack them along the leading dim
#         print("done slicing")

        has_labels = tf.math.reduce_any(tf.reshape(_labels, [-1, IMG_SIZE*IMG_SIZE]) > 0, axis=1)
#         print(has_labels)
        _data = tf.boolean_mask(_data, has_labels)
        _labels = tf.boolean_mask(_labels, has_labels)
        for i in range(_data.shape[0]):
            yield _data[i], _labels[i]

In [5]:
from shiptrack import load_numpy_arrays
all_data, all_labels = load_numpy_arrays(training_dir)

n_test = (all_data.shape[0] // 100) * test_prop
n_val = int((all_data.shape[0]-n_test)*0.181818)  # Fixed validation proportion of ~15% of original dataset
n_train = all_data.shape[0]-n_test-n_val

#     x_test, x_val, x_train = np.split(all_data, [n_test, n_test+n_val])
#     y_test, y_val, y_train = np.split(all_labels, [n_test, n_test+n_val])

#     np.split seems to be reading all the data into memory

x_test, x_val, x_train = all_data[:n_test], all_data[n_test:n_test+n_val], all_data[n_test+n_val:]
y_test, y_val, y_train = all_labels[:n_test], all_labels[n_test:n_test+n_val], all_labels[n_test+n_val:]

print(n_test, 'test samples')
print(n_val, 'val samples')
print(n_train, 'train samples')


110 test samples
380 val samples
1715 train samples


In [6]:
# The labels need to be floats for comparison with the model output
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train.astype('float32')))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val.astype('float32')))


In [7]:
# _data = tf.image.resize(data, INT_IMG_SIZE) / 255.
from tensorflow.keras import layers

def tile(data):
    # Slice the images to the final size...
#     flat_patches = tf.image.extract_patches(images=tf.expand_dims(data, axis=0),
    flat_patches = tf.image.extract_patches(images=data,            
                                            sizes=[1, IMG_SIZE, IMG_SIZE, 1],
                                            strides=[1, IMG_SIZE, IMG_SIZE, 1],  # This should be the same as sizes
                                            rates=[1, 1, 1, 1],
                                            padding='VALID')
    _data = tf.reshape(flat_patches, [-1, IMG_SIZE, IMG_SIZE, 3])  # Stack them along the leading dim
    return _data



resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(*INT_IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])



ORIG_IMG_SIZE = (2030, 1354)

def image_to_patches(image):
    height, width = INT_IMG_SIZE

    image_resized = tf.squeeze(tf.image.resize(image, INT_IMG_SIZE))
    image_reshaped = tf.reshape(image_resized, [height // IMG_SIZE, IMG_SIZE, -1, IMG_SIZE, 3])
    image_transposed = tf.transpose(image_reshaped, [0, 2, 1, 3, 4])
    return tf.reshape(image_transposed, [-1, IMG_SIZE, IMG_SIZE, 3])


def patches_to_image(patches):
    height, width = INT_IMG_SIZE

    image_reshaped = tf.reshape(tf.squeeze(patches), [-1, height // IMG_SIZE, width // IMG_SIZE, IMG_SIZE, IMG_SIZE])
    image_transposed = tf.transpose(image_reshaped, [0, 1, 3, 2, 4])
    image_resized = tf.reshape(image_transposed, [-1, height, width])
    return tf.squeeze(tf.image.resize(tf.expand_dims(image_resized, axis=-1), ORIG_IMG_SIZE), axis=-1)



In [8]:
def test_patching(test_image=1):
    import matplotlib.pyplot as plt
    plt.imshow(x_train[test_image])
    plt.show()
    patches = image_to_patches(x_train[test_image])[..., 0]
    plt.imshow(patches[0])
    plt.show()
    plt.imshow(patches_to_image(patches)[0, ...])
    plt.show()


In [9]:
from keras.layers import Input
from keras.models import Model

# Automatically mirror training across all available GPUs
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
with strategy.scope():

    base_model = Unet(backbone, encoder_weights='imagenet', encoder_freeze=encoder_freeze,
                 classes=1, activation='sigmoid')

    inp = Input(shape=(None, None, 3))
    rescaled = rescale(inp)
    tiled = image_to_patches(rescaled)
    # tiled = tile(resized)
    # augmented = data_augmentation(tiled)
    mod = base_model(tiled)
    out = patches_to_image(mod)
    # I have to use this rather than tf.image.resize because by this point I've lost the channel information
    # out = tf.keras.Sequential([layers.experimental.preprocessing.Resizing(*ORIG_IMG_SIZE)])(repatched)

    model = Model(inp, out, name=base_model.name)

    print(model.summary())

    model.compile(Adam(learning_rate=learning_rate), loss=losses[loss], metrics=[iou_score])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

In [10]:
from keras.callbacks import TensorBoard, ReduceLROnPlateau
tensorboard = TensorBoard(log_dir=tensorboard_dir, histogram_freq=0,
                          write_images=True, write_graph=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=5e-7, verbose=1)

In [11]:

# def get_data_flow(data, labels, subset, batch_size=1):
#     # this is the augmentation configuration we will use for training
#     from keras.preprocessing.image import ImageDataGenerator
#     datagen = ImageDataGenerator(
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         validation_split=0.2)
#     generator = datagen.flow(
#         data, y=labels,
#         batch_size=batch_size if subset == 'training' else 1,
#         subset=subset)
#     return generator

def get_data_flow(data, labels, subset, batch_size=1):
    # we create two instances with the same arguments
    data_gen_args = dict(
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                        )
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    image_datagen.fit(data, augment=True, seed=seed)
    mask_datagen.fit(labels, augment=True, seed=seed)
    image_generator = image_datagen.flow(
        data,
        batch_size=batch_size,
        class_mode=None,
        seed=seed)
    mask_generator = mask_datagen.flow(
        labels,
        batch_size=batch_size,
        class_mode=None,
        seed=seed)
    # combine generators into one which yields image and masks
    train_generator = zip(image_generator, mask_generator)
    return train_generator

if augment:
    raise NotImplemented()

In [12]:
history = model.fit(train_dataset.shuffle(100).batch(2).prefetch(tf.data.AUTOTUNE), validation_data=val_dataset.batch(4).prefetch(tf.data.AUTOTUNE), verbose=1,
                    epochs=epochs, callbacks=[tensorboard, reduce_lr])

Epoch 1/30
INFO:tensorflow:batch_all_reduce: 492 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:batch_all_reduce: 492 all-reduces with algorithm = hierarchical_copy, num_packs = 1
858/858 [==============================] - 813s 823ms/step - loss: nan - iou_score: 0.0018 - val_loss: 1.0138 - val_iou_score: 9.5331e-04
Epoch 2/30
858/858 [==============================] - 688s 800ms/step - loss: nan - iou_score: 0.0026 - val_loss: 1.0195 - val_iou_score: 0.0017
Epoch 3/30
858/858 [==============================] - 689s 802ms/step - loss: nan - iou_score: 0.0108 - val_loss: 1.0313 - val_iou_score: 2.0574e-07
Epoch 4/30
858/858 [==============================] - 691s 803ms/step - loss: nan - iou_score: 0.0323 - val_loss: 1.0488 - val_iou_score: 0.0039
Epoch 5/30
858/858 [==============================] - 688s 801ms/step - loss: nan - iou_score: 0.0525 - val_loss: 1.0493 - val_iou_score: 0.0046
Epoch 6/30
858/858 [==============================] - 687s 799ms/st

In [13]:
# Try augmentation
# Try shuffling the training data

In [14]:
score = model.evaluate(test_ds, verbose=0)

print('Test loss    :', score[0])
print('Test accuracy:', score[1])

NameError: name 'test_ds' is not defined

In [ ]:
# save Keras model for Tensorflow Serving
tf.saved_model.save(
    model,
    os.path.join(model_dir, 'model/1'))